# Querying package metadata with Athena
Quilt stores package data and metadata in S3. Metadata lives in a per-package manifest file
in a each bucket's `.quilt/` directory.


You can therefore query package metadata wth SQL engines like AWS Athena.
Users can write SQL queries to select packages (or files from within packages)
using predicates based on package or object-level metadata.

## Note: Executing Documentation Code  
If you import your AWS credentials for use by `boto3`, you can edit and execute code directly from the notebook version of this document. You can alternatively copy and paste it into your Python editor.

In [3]:
%env aws
%pip install boto3

UsageError: Environment does not have key: aws


This allows you to configure AWS services by calling Python objects:

In [2]:
import boto3,json
SESSION = boto3.session.Session()
print(SESSION)

ATHENA = boto3.client('athena')
IAM = boto3.resource('iam')
S3 = boto3.client('s3')

## I. Granting Access to Athena

By default, Quilt runs with very conservative permissions that do not allow access to [Amazon Athena](https://docs.aws.amazon.com/athena/latest/ug/what-is.html). To enable Athena SQL queries by your Quilt users, you must:

1. Create a new Athena policy.

The standard [AmazonAthenaFullAccess](https://console.aws.amazon.com/iam/home#/policies/arn:aws:iam::aws:policy/AmazonAthenaFullAccess) policy is more permissive than necessary.  For production usage, we recommend creating something more limited, such as:

In [ ]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.ServiceResource.create_policy
AthenaLimitedAccess="""
        "Document": {
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "athena:*"
                    ],
                    "Resource": [
                        "*"
                    ]
                },
                {
                    "Effect": "Allow",
                    "Action": [
                        "glue:GetDatabase",
                        "glue:GetDatabases",
                        "glue:CreateTable",
                        "glue:DeleteTable",
                        "glue:UpdateTable",
                        "glue:GetTable",
                        "glue:GetTables",
                    ],
                    "Resource": [
                        "*"
                    ]
                }
            ]
        }
"""
LimitedAccessPolicy = IAM.create_policy(
    PolicyName='AthenaLimitedAccess',
    PolicyDocument=json.dumps(AthenaLimitedAccess),
    Description='Minimal Athena Access policy for Quilt'
)
print(LimitedAccessPolicy)

2. Attach this policy to your CloudFormation stack.
 
This needs to be done manually by your AWS Administrator:

    a. Go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation)
    b. Select the Quilt stack
    c. Click "Update"
    d. Add the above ARN to the "ManagedUserRoleExtraPolicies" field.
    e. Save
    
3. Add that AWS policy as a Quilt catalog Policy

This needs to be done manually by a Quilt Administrator:

    a. Login to your Quilt instance at, e.g. https://quilt.mycompany.com
    b. Click on "Admin Settings" in the upper right
    c. Scroll down to the "Policies" section on the bottom
    d. Click on the "+" to create a new Policy
    e. Set Title to "AthenaLimitedAccess"
    f. Check "Manually set ARN" and enter ARN of Athena policy.
    f. Click "Create"
    
4. Attach that Policy to a (new) Quilt Role

You cannot attach a policy to the "Custom" Roles, so you will usually need to first create a new Role:

    a. From "Admin Settings", scroll to "Roles"
    b. Click on the "+" to create a new Role
    c. Set Name to e.g., "AthenaAccessRole"
    d. Click on "No policies attached.  Attach a policy..."
    e. Select the "AthenaLimitedAccess" policy from before
    f. Click "Create"

See [Users and roles](../Catalog/Admin.md) for more details on access control management in Quilt.

## II. Create Athena Workgroup and Bucket

Quilt expects a dedicated bucket for Athena queries, which is best to do in its own workgroup

1. Create the Bucket
2. Set an appropriate Policy
3. Create a Workgroup that uses that Bucket



## Defining package tables and views in Athena
The next step in configuring Athena to query the package contents and metadata
is to define a set of tables that represent the package metadata fields as columns.  First, specify the Quilt bucket you want to index:

In [ ]:
QUILT_BUCKET="bucket-name" # without s3:// prefix

BUCKET_ID=QUILT_BUCKET.replace('-','_')
MANIFEST_TABLE=f"{BUCKET_ID}_quilt_manifests"
PACKAGES_TABLE=f"{BUCKET_ID}_quilt_packages"
PACKAGES_VIEW=f"{BUCKET_ID}_quilt_packages_view"
OBJECTS_VIEW=f"{BUCKET_ID}_quilt_objects_view"

### Manifests table
The following Athena DDL will build a table of all the manifests in that bucket
(all package-level and object-level metadata). 

```sql
CREATE EXTERNAL TABLE `quilt_manifests_YOUR_BUCKET`(
  `logical_key` string, 
  `physical_keys` array<string>, 
  `size` string, 
  `hash` struct<type:string,value:string>, 
  `meta` string, 
  `user_meta` string, 
  `message` string, 
  `version` string)
ROW FORMAT SERDE 
  'org.openx.data.jsonserde.JsonSerDe' 
WITH SERDEPROPERTIES ( 
  'ignore.malformed.json'='true') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat'
LOCATION
  's3://{bucket}/.quilt/packages'
TBLPROPERTIES (
  'has_encrypted_data'='false', 
  'transient_lastDdlTime'='1605312102')
```

### Package metadata table
Package names and top hashes are not stored in the manifests. Rather they are stored in pointer files in the `.quilt/named_packages` folder.
The following DDL creates a table from these pointer files to make package
top hashes available in Athena.

```sql
CREATE EXTERNAL TABLE `quilt_named_packages_YOUR_BUCKET`(
  `hash` string)
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://{bucket}/.quilt/named_packages'
TBLPROPERTIES (
  'has_encrypted_data'='false', 
  'transient_lastDdlTime'='1557626200')
```

### View of package-level metadata
The DDL below creates a view that contains package-level information including: 
* User
* Package name
* Tophash
* Timestamp
* Commit message

```sql
CREATE OR REPLACE VIEW "quilt_packages_{bucket}_view" AS
WITH
  npv AS (
    SELECT
      regexp_extract("$path", '^s3:\/\/([^\\/]+)\/([^\/]+)\/([^\/]+)\/([^\/]+)', 4) as user,
      regexp_extract("$path", '^s3:\/\/([^\\/]+)\/([^\/]+)\/([^\/]+)\/([^\/]+)\/([^\/]+)', 5) as name,
      regexp_extract("$path", '[^/]+$') as timestamp,
      "quilt_named_packages_{bucket}"."hash"
      FROM "quilt_named_packages_{bucket}"
  ),
  mv AS (
    SELECT
      regexp_extract("$path", '[^/]+$') as tophash,
        manifest."meta",
        manifest."message"
      FROM
        "quilt_manifests_{bucket}" as manifest
      WHERE manifest."logical_key" IS NULL
  )
SELECT
  npv."user",
  npv."name",
  npv."hash",
  npv."timestamp",
  mv."message",
  mv."meta"
FROM npv
JOIN
  mv
ON
  npv."hash" = mv."tophash" 
```

### View of object-Level metadata
The DDL below creates a view that contains package contents, including:
* logical_key
* physical_keys
* object hash
* object metadata

```sql
CREATE OR REPLACE VIEW "quilt_package_objects_YOUR_BUCKET_view" AS
WITH
  mv AS (
    SELECT
      regexp_extract("$path", '[^/]+$') as tophash,
      manifest."logical_key",
      manifest."physical_keys",
      manifest."size",
      manifest."hash",
      manifest."meta",
      manifest."user_meta"
    FROM
      "quilt_manifests_YOUR_BUCKET" as manifest
    WHERE manifest."logical_key" IS NOT NULL
  )
SELECT
  npv."user",
  npv."name",
  npv."timestamp",
  mv."tophash",
  mv."logical_key",
  mv."physical_keys",
  mv."hash",
  mv."meta",
  mv."user_meta"
FROM mv
JOIN
  "quilt_packages_{bucket}_view" as npv
ON
  npv."hash" = mv."tophash"
```

## Example: query package-level metadata

Suppose we wish to find all .tiff files produced by algorithm version 1.3
with a cell index of 5.

```sql
SELECT * FROM  "quilt_package_objects_YOUR_BUCKET_view" AS
WHERE substr(logical_key, -5)='.tiff'
-- extract and query package-level metadata
AND json_extract_scalar(meta, '$.user_meta.nucmembsegmentationalgorithmversion') LIKE '1.3%'
AND json_array_contains(json_extract(meta, '$.user_meta.cellindex'), '5');
```